In [1]:
pip install openai

In [ ]:
from openai import OpenAI
import random
import pandas as pd

model_id = "gpt-4o"
# model_id = "gpt-4o-mini"
# model_id = "gpt-3.5-turbo"

key = ""
client = OpenAI(
    api_key = key

)

In [14]:
# CSV path (change accordingly)
csv_path = "word_task_stimuli.csv"
total_tokens = 0

# Read CSV
df = pd.read_csv(csv_path)

# creating the columns
for col in ["Model Response", "Order"]:
    if col not in df.columns:
        df[col] = None  # Creates the column but leaves it empty

row_num = 0
# Calling Llama and updating responses
for index, row in df.iterrows():
    stimuli = row["Sentence"]

    response = client.chat.completions.create(
                model=model_id,
                messages = [
                    {"role": "system", "content": "only output the items in the list that match the criteria, in the form of a list [item_1, item_2,...] and nothing else. The items could be words, numbers, emoji sets, and full sentences. Do not include empty characters, or any words such as 'here is the list...'."},
                    {"role": "user", "content": stimuli},
                ])
    gpt_answer = response.choices[0].message.content

    total_tokens += response.usage.total_tokens

    # Store response in the existing DataFrame
    df.at[index, "Model Response"] = gpt_answer
    row_num += 1
    df.at[index, "Order"] = row_num

# Save to csv
csv_output = "word_task_stimuli_gpt_4o.csv"
df.to_csv(csv_output, index=False)

print(f"Messages generated with {total_tokens} tokens")
print(f"Created {csv_output}.")

Messages generated with 41190 tokens
Created word_task_stimuli_gpt_4o.csv.


In [ ]:
# For easier testing without needing to create a new CSV, but doesn't result in CSV output
# Set of examples
statements = [
    'In the list of numbers: 37808945, 77758651, 07640052, 78111153, 09755522, 77524516, which numbers have the digit 8?',
    'In the list of words: acidity, shuffled, occasional, rapped, cropping, funniest, which words have the letter d?',
    "In the list of emoji sets: {'🥺'}, {'🥺', '😭'}, {'😭'}, {'🥺', '😎'}, {'😭', '😎'}, {'😎'}, which sets have the emoji 🥺?"
]

# Generate sentences
for statement in statements:
    messages=[
        {"role": "system", "content": "only output the items in the list that match the criteria, in the form of a list [item_1, item_2,...] and nothing else"},
        {"role": "user", "content": statement},
    ]
    output = pipe(messages, max_new_tokens=256, pad_token_id=128001)
    response = output[0]["generated_text"][-1]["content"]
    print(statement)
    print(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In the list of numbers: 37808945, 77758651, 07640052, 78111153, 09755522, 77524516, which numbers have the digit 8?
[37808945, 77758651, 78111153]
In the list of words: acidity, shuffled, occasional, rapped, cropping, funniest, which words have the letter d?
[shuffled, cropping, occasional]
In the list of emoji sets: {'🥺'}, {'🥺', '😭'}, {'😭'}, {'🥺', '😎'}, {'😭', '😎'}, {'😎'}, which sets have the emoji 🥺?
['🥺'], ['🥺', '😭'], ['🥺', '😎']
